In [2]:
import pandas as pd
import geopandas as gpd
from pathlib import Path

In [ ]:
zcta_path = Path.cwd().parent / 'data' / 'raw' / 'zcta5_data' / 'zcta5.shp'


In [ ]:
/Users/leonardogomes/Library/CloudStorage/GoogleDrive-leodavinci550@gmail.com/My Drive/Elementum/GCB/Data Centers/dc_map_project/data/raw/brasil/infraestrutura_eletrica